In [1]:
import datetime
import time
import pymongo
import re
import requests as rt
import bs4
import json
import numpy as np
import sqlite3 as sq

In [2]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [3]:
def value_tester(index):
    print(product_links[index])
    print(mrp[index])
    print(price[index])
    print(description[index])
    print(gender[index])
    print(brand_names[index])
    print(product_names[index])
    print(primary_image_links[index])
    print(secondary_image_links[index])
    print(sizes[index])

In [4]:
headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'}
ses=rt.Session()
ses.get('https://www.myntra.com/t-shirts',headers=headers)
temp=ses.get('https://www.myntra.com/gateway/v2/search/clothing?f=Gender%3Amen%20women%2Cwomen&p=2&plaEnabled=false&rows=50&o=49',headers=headers)
data_in_json=json.loads(temp.text)
total_items=data_in_json['totalCount']
page_no=int(total_items)//50
p=1
o=0
print('Total pages- ',page_no)
print('Total items-',total_items)

Total pages-  5313
Total items- 265686


In [5]:
#to get product_names, brand_names, image_links,sizes and product_links
a=datetime.datetime.now()
brand_names={}
product_names={}
primary_image_links={}
secondary_image_links={}
product_links={}
sizes={}
mrp={}
price={}
description={}
gender={}
counter=0
for i in range(1,(page_no+1)):
    try:
        p=i
        o=(p*50)-50
        data=ses.get(f'https://www.myntra.com/gateway/v2/search/clothing?f=Gender%3Amen%20women%2Cwomen&p={p}&plaEnabled=false&rows=50&o={p}',headers=headers)
        data_in_json=json.loads(data.text)
        data_in_json=data_in_json['products']
        p+=1
        o+=50
        for j in range(len(data_in_json)):
            brand_names[counter]=data_in_json[j]['brand']
            product_names[counter]=data_in_json[j]['productName']
            product_links[counter]='https://www.myntra.com/'+data_in_json[j]['landingPageUrl']
            primary_image_links[counter]=data_in_json[j]['searchImage']
            mrp[counter]=data_in_json[j]['mrp']
            price[counter]=data_in_json[j]['price']
            gender[counter]=data_in_json[j]['gender']
            description[counter]=data_in_json[j]['additionalInfo']
            temp_data=data_in_json[j]['images']
            temp_links=[]
            temp_size=[]
            temp_size_data=data_in_json[j]['inventoryInfo']
            for w in temp_size_data:
                label,avail=w['label'],w['available']
                if avail== True:
                    temp_size.append(label)
            sizes[counter]=temp_size
            for k in temp_data:
                temp_links.append(k['src'])
            secondary_image_links[counter]=temp_links
            counter+=1
        time.sleep(np.random.randint(1,5))
    except Exception as e :
        print('Exception Occured',e)
        continue
b=datetime.datetime.now()
len(product_links),len(brand_names),len(product_names),len(primary_image_links),len(sizes),len(secondary_image_links),len(mrp),len(price),len(description),len(gender)

Exception Occured 'products'
Exception Occured ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))
Exception Occured ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))
Exception Occured ('Connection aborted.', TimeoutError(10060, 'A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond', None, 10060, None))
Exception Occured HTTPSConnectionPool(host='www.myntra.com', port=443): Max retries exceeded with url: /gateway/v2/search/clothing?f=Gender%3Amen%20women%2Cwomen&p=120

(60027, 60027, 60027, 60027, 60027, 60027, 60027, 60027, 60027, 60027)

In [6]:
#value_tester(100000)

KeyError: 100000

In [8]:
con=sq.connect(r'MyntraWomen.db')  
cur=con.cursor()
cur.execute('CREATE TABLE product_details (Website varchar(40) not null,Product_Link text PRIMARY KEY,Product_Name varchar(50) not null,Product_Brand varchar(50) not null,Size_Avail varchar(20) not null,Price int not null,MRP int not null,Gender varchar(15) not null,Description text not null,Primary_Image_Links text not null,Secondary_Image_Links text not null,Affiliate_Link text not null )')
for i in range(len(product_links)):
    try:
        if i in brand_names.keys() and i in product_names.keys() and i in secondary_image_links.keys() and i in sizes.keys() and i in price.keys() and i in mrp.keys() and i in description.keys() and i in gender.keys()  and i in primary_image_links.keys():
            if brand_names[i] != '' and primary_image_links[i] != ''  and product_names[i] != '' and ','.join(secondary_image_links[i]) != '' and ','.join(sizes[i]) != '' and price[i] != '' and mrp[i] != '' and brand_names[i] != '' and description[i] != '' :
                cur.execute('insert into product_details  values(%r,%r,%r,%r,%r,%d,%d,%r,%r,%r,%r,%r)'%('https://www.myntra.com/',product_links[i],product_names[i],brand_names[i],','.join(set(sizes[i])),price[i],mrp[i],gender[i],description[i],primary_image_links[i],','.join(secondary_image_links[i]),''))
    except:
        pass
con.commit()
con.close()